In [1]:
import os
import cv2
import time
import numpy as np
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input
from keras.applications import *
from keras import regularizers

In [70]:
v_path = r"F:\violent_video_tagging\violence-video-classification\data\raw_frames\test\violent\\"
nv_path = r"F:\violent_video_tagging\violence-video-classification\data\raw_frames\test\non_violent\\"

# v_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\violent\\"
# nv_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\non_violent\\"

In [71]:
violent_frames = []
non_violent_frames = []

for frame in os.listdir(v_path):
    frame = cv2.imread(os.path.join(v_path,frame))
    violent_frames.append(frame)


for frame in os.listdir(nv_path):
    frame = cv2.imread(os.path.join(nv_path,frame))
    non_violent_frames.append(frame)

In [72]:
violent_frames[0].shape

(224, 224, 3)

In [73]:
non_violent_frames[0].shape

(224, 224, 3)

In [65]:
#Creates sequence 
# SHAPE : (N_SAMPLES,TIMESTEP,WIDTH,HEIGHT,CHANNEL)
def create_seq(violent_frames, non_violent_frames):
    
    print("+++ Creating Sequence... +++")
    
    violent_vid = []
    non_violent_vid = []

    i = 0
    while i < len(violent_frames):
        violent_vid.append(violent_frames[i:i+30])
        i = i+30


    i = 0
    while i < len(non_violent_frames):
        non_violent_vid.append(non_violent_frames[i:i+30])
        i = i+30
        
    violent_vid = np.asarray(violent_vid)
    non_violent_vid = np.asarray(non_violent_vid)
    
    return violent_vid, non_violent_vid


In [74]:
def result(x,y):
    
    print("+++ Generating result... +++")
    
    pred = model.predict(x)
#     print('First prediction:', pred)
    
    score = model.evaluate(x, y,verbose=1)
    print("-----------------------------")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    prediction = []
    for p in pred:
        if p>=.5:
            prediction.append(1)
        else:
            prediction.append(0)
    print("-----------------------------")
    print("Classification report")
    print("-----------------------------")
    print(classification_report(y, prediction))
    
    print("-----------------------------")
    print("Confusion Matrix")
    print("-----------------------------")
    conf_mat = confusion_matrix(y, prediction)
    print(conf_mat)
    

In [75]:
def preprocess_lstm(features):
    
    print("+++ Preprocessing data... +++")
    
    violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
    non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]


    print("Violent features: ", violent_features.shape)
    print("Non Violent features: ", non_violent_features.shape)


    violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
    violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

    print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
    print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)


    test_x = np.vstack((violent_vid,non_violent_vid))
    test_y = np.append(violent_y, non_violent_y)

    print("Total data: ", test_x.shape)
    print("Total target: ", test_y.shape)

    test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
    print("(LSTM) After Rehshape: ", test_x.shape)
    
    return test_x,test_y

In [76]:
def feature_extract(violent_frames, non_violent_frames, model):
    
    print("+++ Extracting feature... +++")
    
    violent_frames = np.asarray(violent_frames)
    non_violent_frames = np.asarray(non_violent_frames)

    print ("Before Feature extraction: ")
    print(violent_frames.shape,non_violent_frames.shape)
    all_data = np.vstack((violent_frames,non_violent_frames))
    print("Adding all data: ", all_data.shape)

    #creates feature descriptors
    all_data = all_data.astype('float64')
    desc = preprocess_input(all_data)
    if(model == 'resnet50'):
        loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg19'):
        loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg16'):
        loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
    else:
        print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")
        
    loaded_model = Model(loaded_model.input,loaded_model.output)
    features = loaded_model.predict(desc,batch_size=10,verbose=1)

    print ("After Feature extraction: ", features.shape)
    
    return features

In [77]:
print("+++ Extracting feature... +++")

violent_frames = np.asarray(violent_frames)
non_violent_frames = np.asarray(non_violent_frames)

print ("Before Feature extraction: ")
print(violent_frames.shape,non_violent_frames.shape)
all_data = np.vstack((violent_frames,non_violent_frames))
print("Adding all data: ", all_data.shape)

#creates feature descriptors
all_data = all_data.astype('float64')
desc = preprocess_input(all_data)
model = 'resnet50'
if(model == 'resnet50'):
    loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg19'):
    loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg16'):
    loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
else:
    print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")

+++ Extracting feature... +++
Before Feature extraction: 
(2052, 224, 224, 3) (1872, 224, 224, 3)
Adding all data:  (3924, 224, 224, 3)


In [78]:

loaded_model = Model(loaded_model.input,loaded_model.output)
features = loaded_model.predict(desc,batch_size=10,verbose=1)

print ("After Feature extraction: ", features.shape)

393/393 [==============================] - 309s 786ms/step
After Feature extraction:  (3924, 7, 7, 2048)


In [18]:
import math
30*math.floor(violent_frames.shape[0]/30)

2040

In [19]:
violent_frames.shape[0]

2052

In [22]:
non_violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)

3732

In [27]:
print("+++ Preprocessing data... +++")

violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]

print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)

+++ Preprocessing data... +++
Violent features:  (2040, 7, 7, 2048)
Non Violent features:  (1860, 7, 7, 2048)


In [28]:
violent_frames = violent_features
non_violent_frames = non_violent_features

violent_vid = []
non_violent_vid = []

i = 0
while i < len(violent_frames):
    violent_vid.append(violent_frames[i:i+30])
    i = i+30


i = 0
while i < len(non_violent_frames):
    non_violent_vid.append(non_violent_frames[i:i+30])
    i = i+30

violent_vid = np.asarray(violent_vid)
non_violent_vid = np.asarray(non_violent_vid)

In [30]:
non_violent_vid.shape

(62, 30, 7, 7, 2048)

In [79]:
test_x, test_y = preprocess_lstm(features)

+++ Preprocessing data... +++
Violent features:  (2040, 7, 7, 2048)
Non Violent features:  (1860, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (68, 30, 7, 7, 2048) Non_violent video Seq:  (62, 30, 7, 7, 2048)
Violent Label:  (68,) Non_violent Label:  (62,)
Total data:  (130, 30, 7, 7, 2048)
Total target:  (130,)
(LSTM) After Rehshape:  (130, 30, 100352)


In [80]:
test_x.shape

(130, 30, 100352)

In [81]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50)                20080600  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [82]:
result(test_x,test_y)

+++ Generating result... +++
5/5 [==============================] - 2s 490ms/step - loss: 0.2666 - accuracy: 0.9538
-----------------------------
Test loss: 0.2665787935256958
Test accuracy: 0.9538461565971375
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96        68
         1.0       0.98      0.92      0.95        62

    accuracy                           0.95       130
   macro avg       0.96      0.95      0.95       130
weighted avg       0.96      0.95      0.95       130

-----------------------------
Confusion Matrix
-----------------------------
[[67  1]
 [ 5 57]]
